In [58]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:/Windows/Fonts/malgun.TTF").get_name()
plt.rc("font", family=font_name)

import matplotlib as mlp
mlp.rcParams["axes.unicode_minus"] = False

import urllib.request as req
from urllib.request import urlopen
from urllib.error import HTTPError, URLError
import urllib.parse
from bs4 import BeautifulSoup
import requests
import pprint
import json

from pandas.io.json import json_normalize

# 1. 서울 혼잡빈도강도(읍면동단위)

## 1) Data Prepared

- 서울 혼잡빈도강도 + 동단위 인구(join)
- 서울 혼잡시간강도 + 동단위 인구(join)

In [59]:
# 서울 법정동별 인구현황 데이터 불러오기

pop_seoul_dong = pd.read_csv("data/pop_seoul_dong.csv")
pop_seoul_dong.head()

,자치구,동,인구수,여자,남자
0,종로구,사직동,9630,4335,5295
1,종로구,삼청동,2741,1312,1429
2,종로구,부암동,9702,4654,5048
3,종로구,평창동,18272,8518,9754
4,종로구,무악동,8245,3823,4422


In [3]:
pop_seoul_dong=pop_seoul_dong.set_index(['자치구','동'])

In [4]:
pop_seoul_dong

인구수     여자     남자
자치구 동                        
종로구 사직동    9630   4335   5295
    삼청동    2741   1312   1429
    부암동    9702   4654   5048
    평창동   18272   8518   9754
    무악동    8245   3823   4422
...         ...    ...    ...
강동구 천호2동  33671  16467  17204
    길동    45863  22374  23489
    강일동   32947  15789  17158
    상일1동  38832  19137  19695
    상일2동  10254   4812   5442

[426 rows x 3 columns]

In [61]:
traffic_jam = pd.read_csv("data/2022 서울_혼잡빈도강도_읍면동.csv")
traffic_jam

,행정구역코드,시도명,시군구명,읍면동명,평일주말구분,혼잡빈도강도
0,1101053,서울특별시,종로구,사직동,평일,90.43
1,1101054,서울특별시,종로구,삼청동,평일,32.45
2,1101055,서울특별시,종로구,부암동,평일,60.51
3,1101056,서울특별시,종로구,평창동,평일,45.55
4,1101057,서울특별시,종로구,무악동,평일,36.04
...,...,...,...,...,...,...
3430,3902058,제주특별자치도,서귀포시,서홍동,평일,73.57
3431,3902059,제주특별자치도,서귀포시,대륜동,평일,49.65
3432,3902060,제주특별자치도,서귀포시,대천동,평일,47.58
3433,3902061,제주특별자치도,서귀포시,중문동,평일,56.08


## 2) 혼잡빈도강도(동별)

In [80]:
traffic_jam = traffic_jam[traffic_jam["시도명"] == "서울특별시"]
traffic_jam

KeyError: '시도명'

In [7]:
traffic_jam.drop(["행정구역코드", "시도명", "평일주말구분","시군구명"], axis=1,inplace=True)

In [64]:
traffic_jam = traffic_jam.rename({"읍면동명":"동"}, axis="columns")
traffic_jam

,행정구역코드,시도명,시군구명,동,평일주말구분,혼잡빈도강도
0,1101053,서울특별시,종로구,사직동,평일,90.43
1,1101054,서울특별시,종로구,삼청동,평일,32.45
2,1101055,서울특별시,종로구,부암동,평일,60.51
3,1101056,서울특별시,종로구,평창동,평일,45.55
4,1101057,서울특별시,종로구,무악동,평일,36.04
...,...,...,...,...,...,...
417,1125070,서울특별시,강동구,둔촌1동,평일,21.40
418,1125071,서울특별시,강동구,둔촌2동,평일,28.98
419,1125072,서울특별시,강동구,암사1동,평일,39.04
420,1125073,서울특별시,강동구,천호2동,평일,42.78


In [65]:
traffic_jam=traffic_jam.set_index(["시군구명","동"])

In [67]:
traffic_jam.drop(["행정구역코드", "시도명", "평일주말구분"], axis=1,inplace=True)

In [74]:
traffic_jam.to_csv("data/(처리후)혼잡빈도강도.csv", encoding="utf8)

,자치구,동,인구수,여자,남자
0,종로구,사직동,9630,4335,5295
1,종로구,삼청동,2741,1312,1429
2,종로구,부암동,9702,4654,5048
3,종로구,평창동,18272,8518,9754
4,종로구,무악동,8245,3823,4422
...,...,...,...,...,...
421,강동구,천호2동,33671,16467,17204
422,강동구,길동,45863,22374,23489
423,강동구,강일동,32947,15789,17158
424,강동구,상일1동,38832,19137,19695


In [11]:
traffic_jam_dong = pd.merge(pop_seoul_dong,traffic_jam,how='inner', on="동")

In [68]:
traffic_jam_dong

,인구수,여자,남자,혼잡빈도강도
동,,,,
사직동,9630,4335,5295,90.43
삼청동,2741,1312,1429,32.45
부암동,9702,4654,5048,60.51
평창동,18272,8518,9754,45.55
무악동,8245,3823,4422,36.04
...,...,...,...,...
둔촌2동,24641,12123,12518,28.98
암사1동,34643,17157,17486,39.04
천호2동,33671,16467,17204,42.78


In [13]:
traffic_jam_dong.to_csv("data/traffic_jam_dong.csv", encoding="utf-8")

## 3) 혼잡시간강도 (동별)

In [81]:
traffic_time = pd.read_csv("data/2022 서울_혼잡시간강도_읍면동.csv")
traffic_time.head()

,행정구역코드,시도명,시군구명,읍면동명,평일주말구분,혼잡시간강도
0,1101053,서울특별시,종로구,사직동,평일,95.12
1,1101054,서울특별시,종로구,삼청동,평일,53.03
2,1101055,서울특별시,종로구,부암동,평일,74.40
3,1101056,서울특별시,종로구,평창동,평일,59.39
4,1101057,서울특별시,종로구,무악동,평일,67.78


In [82]:
traffic_time = traffic_time[traffic_time["시도명"] == "서울특별시"]

In [83]:
traffic_time.drop(["행정구역코드", "시도명", "평일주말구분"], axis=1,inplace=True)

In [84]:
traffic_time

,시군구명,읍면동명,혼잡시간강도
0,종로구,사직동,95.12
1,종로구,삼청동,53.03
2,종로구,부암동,74.40
3,종로구,평창동,59.39
4,종로구,무악동,67.78
...,...,...,...
417,강동구,둔촌1동,48.73
418,강동구,둔촌2동,54.40
419,강동구,암사1동,68.03
420,강동구,천호2동,59.84


In [86]:
traffic_time=traffic_time.set_index(["시군구명","읍면동명"])

In [88]:
traffic_time.to_csv("data/(처리후)혼잡시간강도.csv", encoding="utf-8")

In [19]:
traffic_time_dong = pd.merge(pop_seoul_dong,traffic_time,how='inner', on="동")

In [23]:
traffic_time_dong

pandas.core.frame.DataFrame

In [26]:
traffic_jam_dong.to_csv("data/traffic_time_dong.csv", encoding="utf-8")

## 4) 결측치 처리 (전기차 등록 동 기준)

In [48]:
# 구 평균 구하기
traffic_jam_gu = pd.read_csv("data/2022 서울_혼잡빈도강도_읍면동.csv")
traffic_jam_gu.head()

,행정구역코드,시도명,시군구명,읍면동명,평일주말구분,혼잡빈도강도
0,1101053,서울특별시,종로구,사직동,평일,90.43
1,1101054,서울특별시,종로구,삼청동,평일,32.45
2,1101055,서울특별시,종로구,부암동,평일,60.51
3,1101056,서울특별시,종로구,평창동,평일,45.55
4,1101057,서울특별시,종로구,무악동,평일,36.04


In [49]:
traffic_jam_gu = traffic_jam_gu[traffic_jam_gu["시도명"] == "서울특별시"]

In [50]:
traffic_jam_gu.drop(["행정구역코드", "시도명", "평일주말구분"],axis = 1, inplace=True)

In [51]:
traffic_jam_gu

,시군구명,읍면동명,혼잡빈도강도
0,종로구,사직동,90.43
1,종로구,삼청동,32.45
2,종로구,부암동,60.51
3,종로구,평창동,45.55
4,종로구,무악동,36.04
...,...,...,...
417,강동구,둔촌1동,21.40
418,강동구,둔촌2동,28.98
419,강동구,암사1동,39.04
420,강동구,천호2동,42.78


In [41]:
#traffic_jam_gu = traffic_jam_gu.set_index(["시군구명","읍면동명"])

In [44]:
traffic_jam_gu

혼잡빈도강도
시군구명 읍면동명        
종로구  사직동    90.43
     삼청동    32.45
     부암동    60.51
     평창동    45.55
     무악동    36.04
...           ...
강동구  둔촌1동   21.40
     둔촌2동   28.98
     암사1동   39.04
     천호2동   42.78
     길동     73.79

[422 rows x 1 columns]

In [45]:
#grouped = traffic_jam_gu.groupby("시군구명")

In [53]:
traffic_jam_gu_mean=traffic_jam_gu.groupby(traffic_jam_gu["시군구명"]).mean()

In [56]:
traffic_jam_gu_mean = traffic_jam_gu_mean.rename(columns={"혼잡빈도강도":"구평균혼잡빈도강도"})

In [57]:
traffic_jam_gu_mean.to_csv("data/traffic_jam_gu_mean.csv", encoding="utf-8")